https://blog.dominodatalab.com/creating-interactive-crime-maps-with-folium/

ref:
http://python-visualization.github.io/folium/docs-v0.5.0/modules.html
https://python-visualization.github.io/folium/quickstart.html
https://medium.com/@bobhaffner/folium-markerclusters-and-fastmarkerclusters-1e03b01cb7b1 clustering
http://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/Plugins.ipynb plugins

In [25]:
!pip install --upgrade pip
!pip install pandas


Requirement already up-to-date: pip in /opt/conda/lib/python3.8/site-packages (20.2.4)


In [26]:
! pip install folium
import folium as folium

Police_Department_Incident_Reports__2018_to_Present.csv from 
https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-2018-to-Present/wg3w-h783


In [27]:
import pandas as pd
 
#set coordiantes for San Francicso 
sf_coordinates = (37.76, -122.45)

#load crime data for 2018
crimedata = pd.read_csv('Police_Department_Incident_Reports__2018_to_Present.csv')
                        
# create empty map zoomed in on San Francisco
mapsf = folium.Map( location= sf_coordinates , zoom_start=12)




Drop rows that don't have location information in lat / long columns

In [28]:

#drop rows from dataframe with NaN in lat or long
crimedata.dropna(subset=['Longitude'] ,inplace=True)
crimedata.dropna(subset=['Longitude'] ,inplace=True)
print(crimedata.Latitude.isnull().any())
print(crimedata.Longitude.isnull().any())

#reset index 
crimedata.reset_index(inplace=True)

False
False


In [29]:
crimedata.columns

Index(['index', 'Incident Datetime', 'Incident Date', 'Incident Time',
       'Incident Year', 'Incident Day of Week', 'Report Datetime', 'Row ID',
       'Incident ID', 'Incident Number', 'CAD Number', 'Report Type Code',
       'Report Type Description', 'Filed Online', 'Incident Code',
       'Incident Category', 'Incident Subcategory', 'Incident Description',
       'Resolution', 'Intersection', 'CNN', 'Police District',
       'Analysis Neighborhood', 'Supervisor District', 'Latitude', 'Longitude',
       'point'],
      dtype='object')

In [30]:
crimedata.rename(columns={'Police District':'police_district'} , inplace=True)

In [31]:
import folium.plugins

# create a marker cluster called "zone centers"
zone_clusters = folium.plugins.MarkerCluster().add_to(mapsf)


# add a marker for every record in the filtered data, use a clustered view
for i in range (0, 1000 ):
    folium.CircleMarker(radius=5,
        location = [crimedata.Latitude[i], crimedata.Longitude[i]] ,
        popup= crimedata.police_district[i]).add_to(zone_clusters).add_to(zone_clusters)
  

In [32]:
mapsf

 json file download for sf police districts 
https://data.sfgov.org/w/wkhw-cjsf/ikek-yizv?cur=S_J15KQr4Am&from=root
 
 convert the Shapefile into a geojson file.
 http://ogre.adc4gis.com
 The easiest way is to use an ogr2ogr web client. Select the downloaded zip file and put crs:84 in the Target SRS field. Save the result as sfpddistricts.geojson and upload the file to your Domino project

In [33]:
#definition of the boundaries in the map
district_geo = r'/home/jovyan/work/data/sf_police_district_2018.geojson'

In [34]:
import os
print(os.getcwd())
mypath = os.getcwd()

/home/jovyan/work/side_projects


In [35]:
#os.path.join( mypath, 'folium/examples/data/antarctic_ice_shelf_topo.json')

In [36]:
os.path.exists('/home/jovyan/work/data/sf_police_district_2018.geojson')

True

In [37]:
import json
geo_json_data = json.load(open(district_geo))
geo_json_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'shape_area': '91344142.1925',
    'shape_leng': '87550.2751419',
    'company': 'B',
    'shape_le_1': '100231.353916',
    'district': 'SOUTHERN'},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-122.39186138812263, 37.79424680970543],
       [-122.39178965045961, 37.7941712252344],
       [-122.39173396937666, 37.794202997266886],
       [-122.39166681055336, 37.794132183335286],
       [-122.39172264479394, 37.79410052824544],
       [-122.3916726266299, 37.79404791882388],
       [-122.39198184041989, 37.79387175411522],
       [-122.39158906053727, 37.7935276602537],
       [-122.391723729351, 37.79342884992162],
       [-122.39081210865645, 37.79266254752923],
       [-122.38929874837604, 37.791390503319754],
       [-122.38880784444565, 37.790977844242896],
       [-122.38893019202212, 37.790880005297964],
       [-122.38861191205169, 37.790313282442106],
       [-122.3880494246161

In [38]:
# creation of the choropleth
sfmap_pd = folium.Map(location=sf_coordinates, zoom_start=12)
folium.features.GeoJson(district_geo, name = 'Number of incidents per district').add_to(sfmap_pd)
              
display(sfmap_pd)

In [39]:
# calculating total number of incidents per district
crimedata2 = pd.DataFrame(crimedata['police_district'].value_counts().astype(float))
crimedata2

,police_district
Central,16710.0
Mission,15358.0
Southern,13712.0
Northern,13490.0
Tenderloin,11080.0
Bayview,8688.0
Ingleside,7895.0
Taraval,7596.0
Richmond,5479.0
Park,4983.0


In [40]:
crimedata2.iloc[6]

police_district    7895.0
Name: Ingleside, dtype: float64

In [41]:
crimedata2.to_json('crimedis.json')
crimedata2 = crimedata2.reset_index()
crimedata2.columns = ['district', 'Number']
crimedata2

,district,Number
0,Central,16710.0
1,Mission,15358.0
2,Southern,13712.0
3,Northern,13490.0
4,Tenderloin,11080.0
5,Bayview,8688.0
6,Ingleside,7895.0
7,Taraval,7596.0
8,Richmond,5479.0
9,Park,4983.0


In [42]:
crimedata2.district = crimedata2.district.str.upper()

In [43]:
crimedata2

,district,Number
0,CENTRAL,16710.0
1,MISSION,15358.0
2,SOUTHERN,13712.0
3,NORTHERN,13490.0
4,TENDERLOIN,11080.0
5,BAYVIEW,8688.0
6,INGLESIDE,7895.0
7,TARAVAL,7596.0
8,RICHMOND,5479.0
9,PARK,4983.0


In [44]:
# creation of the choropleth
map1 = folium.Map(location=sf_coordinates, zoom_start=12)
map1.choropleth(geo_data = district_geo, 
               
              data = crimedata2,
              columns = ['district', 'Number'],
              key_on = 'feature.properties.district',
              fill_color = 'YlOrRd', 
              fill_opacity = 0.7, 
              line_opacity = 0.2,
              legend_name = 'Number of incidents per district'
              )
map1



/opt/conda/lib/python3.8/site-packages/folium/folium.py:411: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [45]:
from folium import plugins


m = folium.Map(
    location=[35.68159659061569, 139.76451516151428],
    zoom_start=16
)

# Lon, Lat order.
lines = [
    {
        'coordinates': [
            [139.76451516151428, 35.68159659061569],
            [139.75964426994324, 35.682590062684206],
        ],
        'dates': [
            '2017-06-02T00:00:00',
            '2017-06-02T00:10:00'
        ],
        'color': 'red'
    },
    {
        'coordinates': [
            [139.75964426994324, 35.682590062684206],
            [139.7575843334198, 35.679505030038506],
        ],
        'dates': [
            '2017-06-02T00:10:00',
            '2017-06-02T00:20:00'
        ],
        'color': 'blue'
    },
    {
        'coordinates': [
            [139.7575843334198, 35.679505030038506],
            [139.76337790489197, 35.678040905014065],
        ],
        'dates': [
            '2017-06-02T00:20:00',
            '2017-06-02T00:30:00'
        ],
        'color': 'green',
        'weight': 15,
    },
    {
        'coordinates': [
            [139.76337790489197, 35.678040905014065],
            [139.76451516151428, 35.68159659061569],
        ],
        'dates': [
            '2017-06-02T00:30:00',
            '2017-06-02T00:40:00'
        ],
        'color': '#FFFFFF',
    },
]


features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'LineString',
            'coordinates': line['coordinates'],
        },
        'properties': {
            'times': line['dates'],
            'style': {
                'color': line['color'],
                'weight': line['weight'] if 'weight' in line else 5
            }
        }
    }
    for line in lines
]


plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
}, period='PT1M', add_last_point=True).add_to(m)




m

In [46]:
crimedata.rename(columns={'Incident Datetime':'incident_datetime'}, inplace=True)
crimedata.head(1)


,index,incident_datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,...,Incident Description,Resolution,Intersection,CNN,police_district,Analysis Neighborhood,Supervisor District,Latitude,Longitude,point
0,0,2018/01/01 01:30:00 AM,2018/01/01,01:30,2018,Monday,2018/01/01 02:13:00 AM,61870203073,618702,180000263,...,"Robbery, W/ Other Weapon",Open or Active,JUSTIN DR \ COLLEGE AVE,21236000.0,Ingleside,Bernal Heights,9.0,37.732261,-122.423486,"(37.732261252752224, -122.42348641495892)"


In [47]:
from datetime import datetime, timedelta

m1 = folium.Map(sf_coordinates, tiles='stamentoner', zoom_start=6)

hm = folium.plugins.HeatMapWithTime(
        crimedata,
        index=crimedata.incident_datetime,
        auto_play=True
    )

hm.add_to(m1)
m1

In [48]:
import pandas as pd
import numpy as np
import folium.plugins as plugins  
from datetime import datetime, timedelta


    data = [(df3).tolist() for i in range(100)]

    m = folium.Map([40., -73.], tiles='stamentoner', zoom_start=6)

    hm = plugins.HeatMapWithTime(
        data,
        index=time_index,
        auto_play=True
    )

    hm.add_to(m)
    hm.save('map.html')

IndentationError: unexpected indent (<ipython-input-48-ea0fcb2e93b9>, line 7)